In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from docplex.mp.model import Model
from docplex.mp.model import ModelAggregator
from docplex.mp.compat23 import StringIO, izip
from docplex.mp.constants import SOSType, CplexScope, ObjectiveSense
from docplex.mp.constr import AbstractConstraint, LinearConstraint, RangeConstraint, \
    IndicatorConstraint, QuadraticConstraint, PwlConstraint, EquivalenceConstraint

* Preparing the data, arcs, & nodes

In [ ]:
##number of clients##
c=39
#vehile capacity
Cap=3200
#set of nodes
N=[i for i in range(1,c+1)]
#Including Depot
V=[0]+N
#orders to be delivered
file=os.path.abspath("D:/IISE/Vikas 1/Python/orders_Wed.csv")
orders=pd.read_csv(file)
#creating dictionary for the orders and nodes
q={i:orders.iloc[i,2] for i in N}
#storing distance
file=os.path.abspath("D:/IISE/Vikas 1/distance.csv")
d_dummy=pd.read_csv(file)
index=orders["Store ID"]
dummy=(len(index),len(index))
distance=np.zeros(dummy)
i=0
j=0
for i in range(len(index)):
    for j in range(len(index)):
        distance[i][j]=d_dummy.iloc[index[i],index[j]]


loc_x=orders.iloc[:,3]
loc_y=orders.iloc[:,4]
order_dic={i:orders.iloc[i,2] for i in N}
mpl.scatter(loc_x[1:,],loc_y[1:,])
for i in N:
    mpl.annotate(orders.iloc[i,1],(loc_x[i]+3,loc_y[i]))


plt.plot(loc_x[0],loc_y[0],c='r',marker='s')
plt.axis('equal')

#set of arcs
A=[(i,j) for i in V for j in V if i!=j]
cost={(i,j):distance[i][j] for i,j in A}

* Defing Model & Solving

In [ ]:
m= Model('CVRP')
#definig the variables
x=m.binary_var_dict(A,name='x')
u=m.continuous_var_dict(N,ub=Cap,name='u')

#defining the objective
m.minimize(m.sum(cost[i,j]*x[i,j] for i,j in A))

#defining the constraints
#each customer must be visited once
m.add_constraints(m.sum(x[i,j] for j in V if j!=i)==1 for i in N)
m.add_constraints(m.sum(x[i,j] for i in V if i!=j)==1 for j in N)
m.add_indicator_constraints([m.indicator_constraint(x[i,j],u[i]+q[j]==u[j]) for i,j in A if i!=0 and j!=0])
m.add_constraints(u[i]>=q[i] for i in N)
m.parameters.timelimit = 300
#m.parameters.mip.tolerances.mipgap = .40
solution=m.solve(log_output=True)
print(solution)

* Visualize the results

In [ ]:
#collecting the active arcs
active_A = [a for a in A if x[a].solution_value==1]

#printing the network
plt.scatter(loc_x[1:,],loc_y[1:,],c="b",marker=">")
plt.plot(loc_x[0],loc_y[0],c='r',marker="s")
for i, j in active_A:
    plt.plot([loc_x[i],loc_x[j]],[loc_y[i],loc_y[j]],c='g',alpha=0.9,label='route')
plt.show()
